In [1]:
import os
from validation import url_check
from get_info_from_api import github_api_call
from data_processing import (
    document_processing,
    document_chunking,
)
from vector_db import upload_document, get_retriever
from question import get_conversation_chain
from dotenv import load_dotenv

load_dotenv()

github_link = input("GitHub 링크를 입력해주세요 : ")
github_info_dict = github_api_call(github_link, make_txt_file=False)
    
# 3. chunking 후 text processing 하기 
docs = document_processing(github_info_dict, make_txt_file=False)
texts = document_chunking(docs, size=1000, overlap=0)
    
# 4. chunking 된 데이터 vector db 로 임베딩 하기 
db = upload_document(texts, os.getenv('DEEPLAKE_USERNAME'), 'langchain-code')
retriever =  get_retriever(db, "cos", 100, True, 10)
    
# 5. 원하는 질문 입력 하기 
model, qa = get_conversation_chain("gpt-3.5-turbo", retriever)
    
# 6. QA 시작
chat_history = []

Get DIR information.
프로그램 실행 시간: 5.08초
API call 횟수 : 19
OPEN AI deeplake 에 업로드 시작..Deep Lake Dataset in hub://seungeunlee/langchain-code already exists, loading from the storage


Dataset(path='hub://seungeunlee/langchain-code', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
 embedding  embedding  (267, 1536)  float32   None   
    id        text      (267, 1)      str     None   
 metadata     json      (267, 1)      str     None   
   text       text      (267, 1)      str     None   
Deep Lake Dataset in hub://seungeunlee/langchain-code already exists, loading from the storage
  완료! 


In [3]:
import asyncio

async def get_qu(question, chat_history):
    try:
        await asyncio.wait_for(qa({"question": question, "chat_history": chat_history}), timeout=2.0)
    
    except asyncio.TimeoutError:
        print("Time out. ")

In [26]:
chat_history = []
question_list = [
    'what is Transformer?',
    'What is role of encoder?',
    'Please explain Decoder\'s code'
]

# for question in question_list:
#     result = get_qu(question, chat_history)
#     chat_history.append((question, result["answer"]))


In [10]:
chat_history

[('what is Transformer?',
  'Transformer is a model architecture that is widely used in natural language processing tasks, such as machine translation, text generation, and text classification. It was introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017. \n\nThe Transformer model is based on a self-attention mechanism, which allows the model to focus on different parts of the input sequence during processing. This attention mechanism helps the model capture long-range dependencies and improves its performance on tasks that require understanding of context.\n\nThe Transformer model has achieved state-of-the-art results in many NLP tasks and has become a cornerstone in the field of deep learning for natural language processing.'),
 ('What is role of encoder?',
  'The role of the encoder in the Transformer model is to process the input sequence and generate a representation that captures the semantic meaning of the sequence. The encoder consists of multiple layers

In [24]:
from threading import Thread
import functools

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function timeout [%s seconds] exceeded!' % (timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return deco

In [35]:
res = [Exception('function timeout [%s seconds] exceeded!' % (timeout))]
print(res[0])

function timeout [<function timeout at 0x00000189941B6480> seconds] exceeded!


In [36]:
chat_history = []

func = timeout(timeout=5)(qa)


for question in question_list:
    try : 
        result = func(({"question": question, "chat_history": chat_history}))
        chat_history.append((question, result["answer"]))
    except Exception as e:
        pass #print(e)


In [37]:
chat_history

[("Please explain Decoder's code",
  "I'm sorry, but I cannot see the actual code for the Decoder in the provided context. The context only shows some cloning and licensing information related to a repository. If you could provide the actual code for the Decoder or give more specific information, I would be happy to help explain it.")]

In [49]:
docs[0]

Document(page_content='Batch.py', metadata={'content': 'import torch\nfrom torchtext.legacy import data\nimport numpy as np\nfrom torch.autograd import Variable\n\n\ndef nopeak_mask(size, opt):\n    np_mask = np.triu(np.ones((1, size, size)), k=1).astype(\'uint8\')\n    np_mask = Variable(torch.from_numpy(np_mask == 0).to(opt.device))\n    return np_mask\n\ndef create_masks(src, trg, opt):\n    \n    src_mask = (src != opt.src_pad).unsqueeze(-2).to(opt.device)\n\n    if trg is not None:\n        trg_mask = (trg != opt.trg_pad).unsqueeze(-2).to(opt.device)\n        size = trg.size(1)  # get seq_len for matrix\n        np_mask = nopeak_mask(size, opt).to(opt.device)\n        trg_mask = trg_mask & np_mask\n\n    else:\n        trg_mask = None\n    return src_mask, trg_mask\n\n# patch on Torchtext\'s batching process that makes it more efficient\n# from http://nlp.seas.harvard.edu/2018/04/03/attention.html#position-wise-feed-forward-networks\n\nclass MyIterator(data.Iterator):\n    def cre